In [1]:
#from Loader import fileloader as fl
#from Loader import datahandledatabase as dhdb
import ecomis
import pandas as pd
import numpy as np
import time
import posixpath
from datetime import datetime

periodo = 202006

inidb = ecomis.ReadIniFile(mercado="empresas")
TESTPATH = inidb.getTestPath()

dbparser = inidb.getDbIniFileParser()

dbmanager = ecomis.DbDataProcess(str(periodo))
dbmanager.setParser(dbparser)

#To make sqlite dates recognized by external software we follow the ISO-8601 norm so
#date and time in the following format:  YYYY-MM-DD HH:MM:SS.SSS
def textToDateConverter(df, date_cols):
    for datevalue in date_cols:
        df[datevalue] = pd.to_datetime(df[datevalue],format='%Y-%m-%d', dayfirst = True)
    return df

In [13]:
# Equipos quincenales : actualizar
start_time = time.time()

# REPORTERIA ACTIVACIONES VOZ
#********************************************

HALF_MONTHTLY_TEAMS = []
DATE_COLUMNS = ['FECHA_PROCESO','FEC_ACTIV','FEC_DESACTIVA']
ORDER_COLUMN = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM','VENDEDOR',
                'FECHA_PROCESO','ESTADO']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA','COMISION_UNITARIA':'COMISION_UNITARIA_DB'}

COLS = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO','SUPERVISORKAM',
        'VENDEDOR','VENDEDOR_OTRO','POSICION_EMPL',
        'FECHA_PROCESO','CANALDEVENTA','CODIGO','RAZON_SOCIAL','CONTRATO','FEC_ACTIV','FEC_DESACTIVA', 'MODELO','TELEFONO',
        'ESTADO','TMCODE','PLAN_TARIFARIO','DIVISION','FECHA_ACTUALIZACION','TIPODOC','DOCUMENTO','GROSSCOMERCIAL',
        'GROSSCOMISION','ACCESS', 
        'ACCESSBOLSA','ACCESSPAQUETE','ACCESSLICENCIA', 'TIPOEQUIPO', 'RUC_10_20','NEWSADDS','TECNOLOGIAEQUIPO', 'NROORDEN', 
        'ACTIVIDAD','NUEVOSEGMENTO','CEDENTE', 'TIPOVENTA','DEPARTAMENTOGROSS','CARTERIZACION','FACTOR_FACTURACIÓN', 
        'DEALERS_REGIONES', 'COMISION_UNITARIA','PRIMA_SEGURO_ENTEL','ACCESSMASVEP','SEGMENTO_ACCESS','PORTA_EQUIPO']

## Revisar los contratos nulos en BD ##

#"""
ventas_ = dbmanager.loadData('View_Ventas')
ventas_ = textToDateConverter(ventas_, DATE_COLUMNS)
ventas_ = ventas_[COLS]

filter1 = ventas_['PERIODO_ACTIVACION'] == periodo
#filter2 = ventas_['PERIODO_ACTIVACION'] == end_date_dt.month + 201800 # Tener en cuenta diciembre
#filter3 = ventas_['ZONAVENTA'].isin(HALF_MONTHTLY_TEAMS)
#filter4 = ventas_['FECHA_PROCESO'] <= end_date_dt

ventas = ventas_[filter1]
#ventas_1 = ventas_[filter1]  #equipos de quincena
#ventast = ventas_[filter2 & filter3 & filter4] #equipos de quincena
#ventas = ventast_1.append(ventast, ignore_index = True) #equipos de quincena


#Filtros para archivo gce
CHANNEL_FILTER4 = ['SOLUCIONES DE DATOS']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA','DESARROLLO NEGOCIOS','INSIDE SALES']
stage1 = ventas['NEW_GERENCIA2'].isin(CHANNEL_FILTER2)
stage4 = ventas['GERENCIA2'].isin(CHANNEL_FILTER4)

#Filtros para archivo gcp
CHANNEL_FILTER1 = ['J. FERNANDEZ']
CHANNEL_FILTER3 = ['HOGAR']
CHANNEL_FILTER5 = ['DEALER TRADICIONAL','DEALER REMOTO','PLATAFORMAS PRESENCIALES LIMA',
                   'PLATAFORMAS PRESENCIALES REGIONAL','VD PYMES']
stage2 = ventas['GERENCIA1'].isin(CHANNEL_FILTER1)
stage3 = ventas['GERENCIA2'].isin(CHANNEL_FILTER3)
stage5 = ventas['GERENCIA2'].isin(CHANNEL_FILTER5)

#"""
# Filtrando las ventas_gce
#*************************

ventas_gce = ventas[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = ventas_gce['VENDEDOR_OTRO'] != ''
multiple_seller = ventas_gce[filter_reg]['VENDEDOR_OTRO'].values
ventas_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller
ventas_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_gce = ventas_gce.rename(columns = RENAME_COLUMNS)

ventas_gce.sort_values(by = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )

#"""

# Filtrando las ventas_gcp
#*************************

ventas_gcp = ventas[stage2 & ~ stage3 & stage5]
order_col = ORDER_COLUMN.copy()
order_col.pop(0)
ventas_gcp.drop(DROP_COLUMNS2, axis =1, inplace = True)
ventas_gcp.sort_values(by = order_col,ascending = [1,1,1,1,1,1,1,1], inplace = True )

#"""

# Filtrando las ventas_datos
#*************************

ventas_datos = ventas[stage4]
ventas_datos.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_datos = ventas_datos.rename(columns = RENAME_COLUMNS)

ventas_datos.sort_values(by = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )

#"""

# REPORTERIA
ventas_.to_csv(posixpath.join(TESTPATH,str(periodo) + '_ventas_brutas_.csv'), index = False, encoding='utf-8-sig') # Punto de Control
ventas_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_gce.csv'), index = False, encoding='utf-8-sig')
ventas_gcp.to_csv(posixpath.join(TESTPATH ,str(periodo) + '_ventas_gcp.csv'), index = False, encoding='utf-8-sig')
ventas_datos.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_M2M.csv'), index = False, encoding='utf-8-sig')


time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

C:\Users\jsaenza\anaconda\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


--- 6.277971982955933 seconds ---



Notas: 
1) No se consideran registros que tengan algún valor en la columna "NETO".


In [2]:
# REPORTERIA ACTIVACIONES VAS NEW (SSAA)
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA2','ZONA_DE_VENTA',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','ZONA_DE_VENTA','DEPARTAMENTO','SUPERVISORKAM','VENDEDOR_CROSS_SELLING','POSICION_EMPL','ACTION_DATE',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESSREAL',
             'FAMILIA','GANADO_POR','EQUIPO','CUENTA','CONSIDERAR','MI_TIENDA','NETO','OBSERVACIONES']

## Revisar los contratos nulos en BD ##

# Importing, cleaning, filtering and appending == > initial
ventas_ssaa_ = dbmanager.loadData('View_Ventas_SSAA_new')
ventas_ssaa_ = textToDateConverter(ventas_ssaa_, DATE_COLUMNS_DATA) #ok
#ventas_ssaa_ = ventas_ssaa_[COLS_DATA]

ventas_ssaa_before_changes = ventas_ssaa_[COLS_DATA] #Guardamos porsiacaso

# Reemplaza los codigos multiples por un unico login.
filter_reg = ventas_ssaa_['LOGIN_REAL'] != ''
multiple_seller = ventas_ssaa_[filter_reg]['LOGIN_REAL'].values
ventas_ssaa_.loc[filter_reg,'VENDEDOR_CROSS_SELLING'] = multiple_seller
ventas_ssaa_ = ventas_ssaa_[COLS_DATA]

# Filtramos solo los registros del periodo de interes
filter1 = ventas_ssaa_['PERIODO_ACTIVACION'] == periodo
ventas_ssaa = ventas_ssaa_[filter1]

#Filtramos para archivo de grandes clientes
#Se agrego 'DEALERS PYMES REGION' ya que tambien estos deben contar para los gerentes Regionales Sur Centro y Norte.
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA',
                   'DEALER TRADICIONAL','PLATAFORMAS PRESENCIALES REGIONAL','DESARROLLO NEGOCIOS']
stage1 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER2)


#Filtramos para archivo de pymes
CHANNEL_FILTER4 = ['DEALER TRADICIONAL','DEALER REMOTO','PLATAFORMAS PRESENCIALES LIMA', 
                   'PLATAFORMAS PRESENCIALES REGIONAL', 'VD PYMES']
stage2 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER4)



#"""
# Filtrando las ventas_ssaa_gce
#*************************
ventas_ssaa_gce = ventas_ssaa[stage1]
ventas_ssaa_gce.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1], inplace = True )


#"""
# Filtrando las ventas_ssaa_gcp
#*************************

ventas_ssaa_gcp = ventas_ssaa[stage2]
ventas_ssaa_gcp.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1], inplace = True )

#"""
# REPORTERIA

ventas_ssaa.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_new.csv'), index = False, encoding='utf-8-sig') # Para Soluciones de Negocio
ventas_ssaa_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gce_new.csv'), index = False, encoding='utf-8-sig')
ventas_ssaa_gcp.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gcp_new.csv'), index = False, encoding='utf-8-sig')

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

--- 7.811481714248657 seconds ---


C:\Users\jsaenza\anaconda\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jsaenza\anaconda\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
ventas_ssaa_

,PERIODO_ACTIVACION,GERENCIA1,GERENCIA2,ZONA_DE_VENTA,DEPARTAMENTO,SUPERVISORKAM,VENDEDOR_CROSS_SELLING,POSICION_EMPL,ACTION_DATE,CODIGO,...,FECHA_ACTUALIZACION,RUC,ACCESSREAL,FAMILIA,GANADO_POR,EQUIPO,CUENTA,CONSIDERAR,NETO,OBSERVACIONES
0,201902,,CORPORACIONES,CORPORACIONES3,LIMA,,MSANTACRUZ2,GERENTE DE NEGOCIOS VENTAS DIRECTAS,2019-02-01,4.1994.00.00.100006,...,2019-03-14 03:16:39,20422293699,10.170000,MDM,MALTAMIRANO2,HARCOS,1,NO VENTA MES PASADO,,No Considerar
1,201902,,CORPORACIONES,CORPORACIONES3,LIMA,,MSANTACRUZ2,GERENTE DE NEGOCIOS VENTAS DIRECTAS,2019-02-01,4.1994.00.00.100006,...,2019-03-14 03:16:39,20422293699,10.170000,MDM,MALTAMIRANO2,HARCOS,1,NO VENTA MES PASADO,,No Considerar
2,201902,,CORPORACIONES,CORPORACIONES3,LIMA,,MSANTACRUZ2,GERENTE DE NEGOCIOS VENTAS DIRECTAS,2019-02-01,4.1994.00.00.100006,...,2019-03-14 03:16:39,20422293699,10.170000,MDM,MALTAMIRANO2,HARCOS,1,NO VENTA MES PASADO,,No Considerar
3,201902,,CORPORACIONES,CORPORACIONES3,LIMA,,MSANTACRUZ2,GERENTE DE NEGOCIOS VENTAS DIRECTAS,2019-02-01,4.1994.00.00.100006,...,2019-03-14 03:16:39,20422293699,10.170000,MDM,MALTAMIRANO2,HARCOS,1,NO VENTA MES PASADO,,No Considerar
4,201902,,VENTA REGIONAL EMPRESA,REGIONAL NORTE,PIURA,CARTERA NORTE,MVALDEZ2,EJECUTIVO DE REGIONES,2019-02-01,5.15887.00.00.100002,...,2019-03-14 03:16:39,20525474071,12.710000,WAP1,REGIONES,HARCOS,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30882,202005,,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,JUNIN,CAPTURA CENTRO,ECORNEJO2,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-05-30,5.29718.00.00.100000,...,2020-06-10 16:48:57,20486428709,253.389831,O365,GCABA2,AMENDIETA,23,,,
30883,202005,,VD PYMES,WALLA,LIMA,,KMEJIAL,CONSULTOR VENTAS DIRECTAS,2020-05-31,,...,2020-06-10 16:48:57,20519043468,51.694915,LICENCIA,XFRANCO,IKAIKA,5,,,
30884,202005,,VD PYMES,MICHA,LIMA,,GQUINONES,CONSULTOR VENTAS DIRECTAS,2020-05-29,5.29710.00.00.100001,...,2020-06-10 16:48:57,20602633846,231.400000,GOOGLE,ACERVANTES,IKAIKA,13,,,
30885,202005,,VD PYMES,KRATOS,LIMA,,JVALERIO,CONSULTOR VENTAS DIRECTAS,2020-05-30,5.29721.00.00.100001,...,2020-06-10 16:48:57,20100583845,267.000000,GOOGLE,ACERVANTES,IKAIKA,15,,,


In [3]:
ventas_ssaa_gce

,PERIODO_ACTIVACION,GERENCIA1,GERENCIA2,ZONA_DE_VENTA,DEPARTAMENTO,SUPERVISORKAM,VENDEDOR_CROSS_SELLING,POSICION_EMPL,ACTION_DATE,CODIGO,...,FECHA_ACTUALIZACION,RUC,ACCESSREAL,FAMILIA,GANADO_POR,EQUIPO,CUENTA,CONSIDERAR,NETO,OBSERVACIONES


In [ ]:
# REPORTERIA ACTIVACIONES VAS (SSAA) OLD
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

#half_monthly_teams = ['MICHA','KRATOS','KAIZEN']
half_monthly_teams = []
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS_DATA ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA'}

COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO',
             'SUPERVISORKAM','VENDEDOR_CROSS_SELLING','VENDEDOR_OTRO','POSICION_EMPL','ACTION_DATE','CANALDEVENTA',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESS_REAL',
             'TIPO2','GANADO_POR','EQUIPO','CUENTA','OBSERVACIONES']

## Revisar los contratos nulos en BD ##

# Importing, cleaning, filtering and appending == > initial
ventas_ssaa_ = dbmanager.loadData('View_Ventas_SSAA')
ventas_ssaa_ = textToDateConverter(ventas_ssaa_, DATE_COLUMNS_DATA)
ventas_ssaa_ = ventas_ssaa_[COLS_DATA]

filter1 = ventas_ssaa_['PERIODO_ACTIVACION'] == periodo
filter2 = ventas_ssaa_['PERIODO_ACTIVACION'] == end_date_dt.month + 201800 # Tener en cuenta diciembre
filter3 = ventas_ssaa_['ZONAVENTA'].isin(half_monthly_teams)
filter4 = ventas_ssaa_['ACTION_DATE'] <= end_date_dt

ventas_ssaa_t_1 = ventas_ssaa_[filter1]
ventas_ssaa_t = ventas_ssaa_[filter2 & filter3 & filter4]
ventas_ssaa = ventas_ssaa_t_1.append(ventas_ssaa_t, ignore_index = True)

stage1 = ventas_ssaa['NEW_GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = ventas_ssaa['GERENCIA1'].isin(CHANNEL_FILTER1)
stage3 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER3)

#"""
# Filtrando las ventas_ssaa_gce
#*************************
ventas_ssaa_gce = ventas_ssaa[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = ventas_ssaa_gce['VENDEDOR_OTRO'] != ''
multiple_seller = ventas_ssaa_gce[filter_reg]['VENDEDOR_OTRO'].values
ventas_ssaa_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller
ventas_ssaa_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_ssaa_gce = ventas_ssaa_gce.rename(columns = RENAME_COLUMNS_DATA)
ventas_ssaa_gce.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )


#"""

#"""
# Filtrando las ventas_ssaa_gcp
#*************************

ventas_ssaa_gcp = ventas_ssaa[stage2 & ~ stage3]
ventas_ssaa_gcp.drop(DROP_COLUMNS2, axis =1, inplace = True)
ventas_ssaa_gcp.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )

#"""

# REPORTERIA


ventas_ssaa_.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_.csv'), index = False, encoding='utf-8-sig') # Para Soluciones de Negocio
ventas_ssaa_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gce.csv'), index = False, encoding='utf-8-sig')
ventas_ssaa_gcp.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gcp.csv'), index = False, encoding='utf-8-sig')

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)